In [14]:
# -*- coding: utf-8 -*-

#Check Cuda Version
#!nvcc --version

#!pip uninstall pytorch
#!pip uninstall pytorch-nightly
#!pip uninstall torchvision

#Download Cuda 9.2 and PyTorch 0.4.1
#!git clone https://gist.github.com/f7b7c7758a46da49f84bc68b47997d69.git
#!bash /content/f7b7c7758a46da49f84bc68b47997d69/pytorch041_cuda92_colab.sh



#!pip3 install https://download.pytorch.org/whl/cu100/torch-1.0.1.post2-cp36-cp36m-linux_x86_64.whl
#!pip3 install torchvision

#!pip3 install --upgrade torch torchvision

import torch
!nvcc --version
print(torch.__version__)

from __future__ import print_function, division


import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
import cv2
from PIL import Image, ImageOps, ImageEnhance, PILLOW_VERSION


#import fastai


class adjust_brightness(object):
    def __init__(self, brightness_factor):
      self.brightness_factor = brightness_factor
    
    def __call__(self, img):
      enhancer = ImageEnhance.Brightness(img)
      img = enhancer.enhance(self.brightness_factor)
      return img    


#print(torchvision.models.resnet.model_urls)
# Data augmentation and normalization for training
# Just normalization for validation
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

data_transforms2 = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        #transforms.ColorJitter(brightness=0.6, contrast=0, saturation=0, hue=0),
        #transforms.RandomGrayscale(p=1),
        adjust_brightness(0.65),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        adjust_brightness(0.8),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}




data_dir = '/content/drive/My Drive/Colab Notebooks/notebook/catdog_data'
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=16,
                                             shuffle=True, num_workers=16)
              for x in ['val']}
image_datasets2 = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms2[x])
                  for x in ['train', 'val']}
dataloaders2 = {x: torch.utils.data.DataLoader(image_datasets2[x], batch_size=16,
                                             shuffle=True, num_workers=16)
              for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['val']}
class_names = image_datasets['val'].classes

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


##############################################################################################
path='/content/drive/My Drive/Colab Notebooks/notebook/CNN_dogcat0810_ResNet50_dict2.pt'
##############################################################################################


if torch.cuda.is_available():
    #test_model = torch.load(path)
    test_model = models.resnet50(pretrained=False)
    #test_model = torch.hub.load('moskomule/senet.pytorch', 'se_resnet50', pretrained=False,)
    num_ftrs = test_model.fc.in_features
    test_model.fc = nn.Linear(num_ftrs, 2)
    test_model.load_state_dict(torch.load(path))
    test_model.eval()
    test_model.cuda()
else:
    device = torch.device('cpu')
    #test_model = models.resnet50(pretrained=False)
    test_model = torch.hub.load('moskomule/senet.pytorch', 'se_resnet50', pretrained=False,)
    num_ftrs = test_model.fc.in_features
    test_model.fc = nn.Linear(num_ftrs, 2)
    test_model.load_state_dict(torch.load(path, map_location=device))
    test_model.eval()
print('The model is loaded')



# class TempModel(nn.Module):
#     def __init__(self):
#         self.conv1 = nn.Conv2d(3, 5, (3, 3))
#     def forward(self, inp):
#         return self.conv1(inp)

# model = TempModel()
# model.load_state_dict(torch.load(file_path))
# model.eval()



correct = 0.0
total = 0.0
since = time.time()


since = time.time()
print('Evaluation start time is {}'.format(time.strftime('%c', time.localtime(since+32400))))

with torch.no_grad():
  for i, (inputs, labels) in enumerate(dataloaders['val']):
    inputs = inputs.to(device)
    labels = labels.to(device)
    outputs = test_model(inputs)
    _, preds = torch.max(outputs, 1)
    total += labels.size(0)
    correct += (preds == labels).sum().item()
  for i, (inputs, labels) in enumerate(dataloaders2['val']):
    inputs = inputs.to(device)
    labels = labels.to(device)
    outputs = test_model(inputs)
    _, preds = torch.max(outputs, 1)
    total += labels.size(0)
    correct += (preds == labels).sum().item()
    #print(correct)
time_elapsed = time.time() - since
print('Evaluation complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
print('Accuracy of the network on the whole test images: %.4f %%' % (
    100 * correct / total))

# with torch.no_grad():
#   for inputs, labels in dataloaders['val']:
#     inputs = inputs.to(device)
#     labels = labels.to(device)
#   outputs = net(images)
#   _, predicted = torch.max(outputs.data, 1)
#   total += labels.size(0)
#   correct += (predicted == labels).sum().item()
#   print(correct)








######################################################################
# Visualize a few images
# ^^^^^^^^^^^^^^^^^^^^^^
# Let's visualize a few training images so as to understand the data
# augmentations.

# def imshow(inp, title=None):
#     """Imshow for Tensor."""
#     inp = inp.numpy().transpose((1, 2, 0))
#     mean = np.array([0.485, 0.456, 0.406])
#     std = np.array([0.229, 0.224, 0.225])
#     inp = std * inp + mean
#     inp = np.clip(inp, 0, 1)
#     plt.imshow(inp)
#     if title is not None:
#         plt.title(title)
#     plt.pause(0.001)  # pause a bit so that plots are updated


# # Get a batch of training data
# inputs, classes = next(iter(dataloaders['train']))

# # Make a grid from batch
# out = torchvision.utils.make_grid(inputs)

# # imshow(out, title=[class_names[x] for x in classes])


######################################################################
# Training the model
# ------------------
#
# Now, let's write a general function to train a model. Here, we will
# illustrate:
#
# -  Scheduling the learning rate
# -  Saving the best model
#
# In the following, parameter ``scheduler`` is an LR scheduler object from
# ``torch.optim.lr_scheduler``.


def evaluate_model(model, criterion, num_epochs=1):
    since = time.time()

    #best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['val']:
            model.eval()   # Set model to evaluate mode
            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                #best_model_wts = copy.deepcopy(model.state_dict())


    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    # model.load_state_dict(best_model_wts)
    return model


######################################################################
# Visualizing the model predictions
# ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
#
# Generic function to display predictions for a few images
#

# def visualize_model(model, num_images=6):
#     was_training = model.training
#     model.eval()
#     images_so_far = 0
#     fig = plt.figure()

#     with torch.no_grad():
#         for i, (inputs, labels) in enumerate(dataloaders['val']):
#             inputs = inputs.to(device)
#             labels = labels.to(device)

#             outputs = model(inputs)
#             _, preds = torch.max(outputs, 1)

#             for j in range(inputs.size()[0]):
#                 images_so_far += 1
#                 ax = plt.subplot(num_images//2, 2, images_so_far)
#                 ax.axis('off')
#                 ax.set_title('predicted: {}'.format(class_names[preds[j]]))
#                 imshow(inputs.cpu().data[j])

#                 if images_so_far == num_images:
#                     model.train(mode=was_training)
#                     return
#         model.train(mode=was_training)

######################################################################
# Finetuning the convnet
# ----------------------
#
# Load a pretrained model and reset final fully connected layer.
#

"""Original Model Used"""
"""with pretrained=Flase, you'll need much, much 
more epochs(often in hundreds) to get a good result"""
#model_ft = models.resnet18(pretrained=True)

# New Model Used
model_ft = test_model

# num_ftrs = model_ft.fc.in_features
# model_ft.fc = nn.Linear(num_ftrs, 2)

# model_ft = model_ft.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimizedCNN_dogcat0810.pt

# Original Optimizer Used
#optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

# New Optimizer Used_Performance Very Unsatisfactory
#optimizer_ft = optim.Adam(model_ft.parameters(), lr=0.00001, betas = (0.9, 0.999), eps=1e-8)

# Decay LR by a factor of 0.1 every 4 epochs
# exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=8, gamma=0.1)

"""Use fixed LR for comparison"""

#learn = create_cnn(image_datasets, models.resnet50, metrics=accuracy)
#learn.lr_find()
#learn.recorder.plot()

######################################################################
# Train and evaluate
# ^^^^^^^^^^^^^^^^^^
#
# It should take around 15-25 min on CPU. On GPU though, it takes less than a
# minute.
#

#model_ft = evaluate_model(model_ft, criterion, num_epochs=1)



######################################################################
#

#visualize_model(model_ft)

# Original code for saving
#torch.save(model_ft, 'CNN_dogcat0810_ResNet50.pt')

# To resolve errors in CNN_test
#torch.save(model_ft.state_dict(), save_path)
print('Evaluation Complete')

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2018 NVIDIA Corporation
Built on Sat_Aug_25_21:08:01_CDT_2018
Cuda compilation tools, release 10.0, V10.0.130
1.2.0
The model is loaded
Evaluation start time is Mon Oct 14 17:44:51 2019
Evaluation complete in 2m 59s
Accuracy of the network on the whole test images: 99.3204 %
Evaluation Complete


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive
